In [1]:
import requests
import bs4
import dataclasses
import typing
import psycopg2
import os

get a list of all athletes

In [2]:
DB_URL=os.getenv("DB_URL")
connection = psycopg2.connect(
    DB_URL,
)

In [3]:
@dataclasses.dataclass
class Athlete:
    name: str
    nickname: str
    url: str

In [4]:
def get_urls():
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "SELECT url"
                " FROM url;"
            ),
        )
        rows = cursor.fetchall()
    return [row[0] for row in rows]

In [5]:
existing_urls = get_urls()

In [6]:
def get_athletes_from_source(athlete_urls: typing.List[str]):
    result = []
    def row_to_athlete(row: typing.List[bs4.element.Tag]):
        name = f"{data[0].text} {data[1].text}"
        name = name.replace("  ", " ")
        return Athlete(
            name=name,
            nickname=data[2].text,
            url=f"https://www.bjjheroes.com{data[0].find('a').get('href')}",
        )
    res = requests.get("https://www.bjjheroes.com/a-z-bjj-fighters-list")
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    table = soup.find_all('tr')
    for row in table:
        data = row.find_all('td')
        if data:
            a = row_to_athlete(data)
            if a.url not in athlete_urls:
                result.append(a)
    return result

In [7]:
athletes = get_athletes_from_source(existing_urls)

In [8]:
len(athletes)

1

add athletes to athlete table:

In [15]:
def add_athlete_to_db(athlete: Athlete):
    with connection.cursor() as cursor:
        cursor.execute(
            (
                "INSERT INTO athlete (name, nickname)"
                " VALUES (%s, %s)"
                " RETURNING id"
            ),
            (athlete.name, athlete.nickname),
        )
        id_row = cursor.fetchone()
        id = id_row[0]
        
        cursor.execute(
            (
                "INSERT INTO url (url, athlete_id)"
                " VALUES (%s, %s)"
            ),
            (athlete.url, id),
        )
        connection.commit()

In [16]:
for athlete in athletes:
    add_athlete_to_db(athlete)